# A Python puzzle: Using a series of increasing default values for defaultdict

This page came out of a simple little coding puzzle Aaron Maxwell, author of [_Powerful Python_](https://powerfulpython.com/) had sent to his mailing list: how to use default of 1 with a Python defaultdict.  I answered this with `defaultdict(lambda: 1)`, and then I thought of making this a bit harder and coming up with a way to make the default value non-constant, for example a sequence.  This turned into an interesting exploration of callables and persisting state between calls (without using global variables, of course) in the Python language.

## Background: dict and defaultdict

The Python `dict` type (_associative array_ or _hash_ in some other languages) has the semantics that if you try to look up a key that doesn't exist it raises a _KeyError_ exception.  This is fine for many use cases, but it makes some other uses cases unnecessarily complicated.   An example is using a dict to count the number of occurences of individual words in some source text; the following would be a concise and simple solution, but doesn't work...

In [1]:
source_text = "spam eggs spam and spam"

d = dict()
for word in source_text.split():
    # d[word] += 1   # on first word this would raise KeyError.
    pass

To make this work we could test if the word is already in `d`, and if it isn't intialize its associated value to 0.
But there is a type in the Python standard library that does exactly this automatically... the [`collections.defaultdict`](https://docs.python.org/3/library/collections.html#collections.defaultdict) class is a `dict` which on looking up a non-existing key calls a user-provided _initializer_ _callable_ (which the manual entry calls a "default factory") instead of raising an exception...

In [2]:
from collections import defaultdict

dd = defaultdict(int)
for word in source_text.split():
    dd[word] += 1
for (key, count) in dd.items():
    print(f'{key} occurred {count} times')


spam occurred 3 times
eggs occurred 1 times
and occurred 1 times


Here we're using `int` as the default factory because calling `int()` without arguments returns zero... we might also have used `defaultdict(lambda: 0)`; we can't just pass `0` directly because the default factory has to be a callable object (i.e. a function, method, type constructor or anything else that provides a `__call__` method) which _returns_ the default we want.

## The Puzzle

Now back to our puzzle.  What can we pass to defaultdict as a default factory so that our default values will be serialized rather than constant? (i.e. 1 for the first key failed lookup, 2 for the second, etc.)  Let's come up with all possible _distinct_ solutions to this question without using a global variable (and without doing any meta-programming, i.e. just using the unaltered Python language.)

So far we've discovered 5 distinct solutions to this in Python 3.  If you find any others, by all means send them to me.

## 1) Using a counter object

The argument we need to pass to defaultdict has to be a callable, and we need to keep state between calls (tracking the count), so since Python is an objrect-oriented language that suggests an object (state) with a method (callable).  A simple counter object would seem to be the most obvious.  The object's class might define a `nextcount()` method, but since it does nothing else we might was well just put the counting code in the `__call__` [dunder](https://wiki.python.org/moin/DunderAlias) method, making the object itself callable.

In [3]:
class CounterObj:
    def __init__(self):
        self.count = 0
    def __call__(self):
        self.count += 1
        return self.count

dd = defaultdict(CounterObj())
print(dd['spam'], dd['eggs'], dd['spam'], dd['and'], dd['spam'])

1 2 1 3 1


## 2) Subclassing 'int' to instantiate a serialized default integer

The manual entry for defaultdict gives the example of `defaultdict(int)` for default elements of 0. This works because 0 is the default `int`, i.e. what the int class (the int constructor) returns when it's called without arguments.  So we should be able to create a subclass of `int` that makes a serialized default instance instead.  This turns out to be pretty easy, by extending the `__new__` dunder method and keeping track of the count in a class attribute.  A `SerialInt` is a normal integer except that its default is a serial number rathern than 0.

In [4]:
class SerialInt(int):
    count = 0
    def __new__(self, x=None, **kwargs):
        self.count += 1
        return(int.__new__(self, self.count if x is None else x, **kwargs))

dd = defaultdict(SerialInt)
print(dd['spam'], dd['eggs'], dd['spam'], dd['and'], dd['spam'])

1 2 1 3 1


It should be noted that unlike in solution #1 which has a separate counter object for every defaultdict, the count here is a global state, i.e. it keeps counting across multiple defaultdicts because it's a class attribute and there is only one SerialInt class object, thus making this an instance of the [singleton](https://en.wikipedia.org/wiki/Singleton_pattern) pattern.

## 3) Using an iterator or generator

Although generally considered the most "Pythonic" way of producing a sequence, and first look iterators or generators might not seem to be a great match for this particular problem because they aren't in themselves callable.  But there are some ways around this; one is that we could "cheat" and pass the `__next__` dunder method of the generator as the default factory!  (This might be called "cheating" because dunder methods are not intended to be called or passed directly.)

In [5]:
def seq_generator_func():
    count = 1
    while True:
        yield count
        count += 1

dd = defaultdict(seq_generator_func().__next__)
print(dd['spam'], dd['eggs'], dd['spam'], dd['and'], dd['spam'])

1 2 1 3 1


What's more, there is a counter generator already in the standard library, so we don't even have to code it... giving us a "one-liner" solution to our puzzle if you don't count the import statement.

In [6]:
from itertools import count

dd = defaultdict(count(1).__next__)
print(dd['spam'], dd['eggs'], dd['spam'], dd['and'], dd['spam'])

1 2 1 3 1


In both of these cases we're calling a generator funtion to instantiate a generator object, and then passing the `__next__` method of that generator instance object to the defaultdict.  When you pass a bound method (as opposed to an unbound method which in Python can be referred to by `Classname.method`) the object it is bound to goes along for the ride, just like when the method is invoked, so this instantiated count generator becomes our default factory.  In essence in Python a bound method is a "closure" over its object... which is a nice segue to our next solution.

But first, there is another way we can get a callable to use with a generator that avoids having to reference the `__next__` method; the functools standard library provides the function `partial()`, which takes a function and some arguments and returns a new function that is [partially applied](https://en.wikipedia.org/wiki/Partial_application) to those arguments.  With help from this we can use the perhaps more appropriate `next()` function...

In [7]:
from functools import partial

dd = defaultdict(partial(next, count(1)))
print(dd['spam'], dd['eggs'], dd['spam'], dd['and'], dd['spam'])

1 2 1 3 1


This one-liner solution to the puzzle was posted by Reddit user [u/preoccupied_siege](https://www.reddit.com/user/preoccupied_siege).

## 4) Using a lexical closure

Lexical closures are a way for a function object to maintain state long-term that's heavily used in Scheme and some other Lisp-y or functional languages, not so much in Python where objects are usually preferred.  But at least in Python 3 it is possible to solve this puzzle with a lexical closoure, although there are some gotchas... specifically, we need to use a [`nonlocal`](https://docs.python.org/3/reference/simple_stmts.html#grammar-token-nonlocal-stmt) declaration for the count variable.  Also, in Scheme the inner function would usually be a lambda since it really doesn't need a name, but in Python lambdas are deliberately limited to avoid them having side effects (like updating `count`).  I dug a bit into the reasons for this limitation, and without going into all the details, what it comes down to is that Guido van Rossum wants to encourage a very explicit style in Python and thinks that in all but the most trivial cases a named function is  always preferrable to a lambda.  In fact, at one point he intended to [remove lambda from the language altogether](https://www.artima.com/weblogs/viewpost.jsp?thread=98196). Upon reflection I think that the limited lambdas are fine for Python; if I want to code in a functional style I'll use a fully functional language.  And anyone who really wants code Python in a fully functional style can try [Coconut](coconut-lang.org)!  But it's good to know that we do have the functionality of lexical closures available in plain Python and of course Python functions are still first class objects, even if they do have to have a name.

In [8]:
def make_counter():
    count = 0
    def counter():
        nonlocal count
        count += 1
        return count
    return counter

dd = defaultdict(make_counter())
print(dd['spam'], dd['eggs'], dd['spam'], dd['and'], dd['spam'])

1 2 1 3 1


Here, when we call `make_counter()` it returns a reference to the inner `counter` function packaged together with the `count` variable from `make_counter`'s scope... we say that the function reference "is closed over" the count variable, which has now gone out of its original scope because `make_counter` is done, but continues to exist because it's referenced in `counter`'s `__closure__` function attribute.  Since we're going to modify this variable we have to use `nonlocal` to avoid creating a new local binding instead forcing a closure, but otherwise this works the same as a lexical closure would in most other languages that have them.

However, [this isn't always the case](http://ashtonkemerling.com/blog/2013/04/30/binding-vs-assignment/)... because of Python's different scoping rules in some cases trying to create a lexical closure produces results programmers used to other languages might find surprising.

## 5) Using a mutable default argument to maintain state

Novice Python programmers often trip over this (see i.e. this [page about Python gotchas](https://docs.python-guide.org/writing/gotchas)) but another way for a function to maintain state between invocations in Python is via a mutable default argument.  This is a consequence of the fact that in Python arguments are always passed by reference combined with the fact that Python evaluates default arguments at function definition time.  So if a default argument is a mutable object, for example a list, as in `def f(l=[])`, then it will be created only once and any changes made to it will persist between calls (as opposed to, in this case, `l` being re-initialized to an empty list on each call, which is what the programmer might have intended).  This is correct behavior for Python, but can be surprising when you first encounter it.

In [9]:
def counter(count=[0]):
    count[0] += 1
    return count[0]

dd = defaultdict(counter)
print(dd['spam'], dd['eggs'], dd['spam'], dd['and'], dd['spam'])

1 2 1 3 1


Here the state of our counter is encapsulated in a list element to bind the `count` variable to a mutable object (list) rather than it being rebound every time we update it as a immutable argument (like an Integer) would be.  As in solution #2, and unlike the other 3 solutions, this is a singleton, i.e. there will be only one counter for any number of defaultdicts.  Although it may seem like we're abusing some unintended behavior, in some cases this may actually be more readable and/or predictable than a real lexical closure, and therefor might be preferred.  Like a lexical closure a mutable default argument provides a strong form of encapsulation (by Python standards)... if the mutable object is created as part of the function's definition, then only that function can change it.  A caller can temporarily _override_ it, but they can't _overwrite_ it!  Well, at least not without digging around in function attributes... but then we'd be going down the rabbit hole of meta-programming, where anything is possible.

In [10]:
def pile_it_on(item, pile=[]):
    pile.append(item)
    return pile

print(pile_it_on("spam"))
print(pile_it_on("spam"))
print(pile_it_on("eggs", []))  # I'd like just eggs, hold the spam...
print(pile_it_on("spam"))      # surprise: no more eggs, just more spam!


['spam']
['spam', 'spam']
['eggs']
['spam', 'spam', 'spam']


In Python, mutable default arguments would appear to be a useful [spandrel](https://en.wikipedia.org/wiki/Spandrel_%28biology%29), providing a level of encapsulation similar to lexical closures while being potentially more predictable because the scope of the variable is explicitly tied to the function it's an argument of.

Before concluding; some Reddit users found ways of turning this solution into one-liners; this involves sidestepping the aforementioned limitations of lambda, which, like nearly everything in Python, is possible if you really insist...

In [12]:
# posted by u/FAVORED_PET
dd = defaultdict(lambda c=enumerate(iter(int, 1), 1): next(c)[0])
print(dd['spam'], dd['eggs'], dd['spam'], dd['and'], dd['spam'])

# posted by u/yawpitch
dd = defaultdict(lambda c=[0]: c.append(c.pop() + 1) or c[0])
print(dd['spam'], dd['eggs'], dd['spam'], dd['and'], dd['spam'])


1 2 1 3 1
1 2 1 3 1


# Conclusion

If we make an effort to understand everything thoroughly, then even solving a simple little puzzle like this one can turn into a deep exploration of a programming language.  Here we've used callable objects and class objects, beat a square  generator into a round hole, and explored some more esoteric ways to encapsule persistent state.   Implementing a simple lexical closure in Python requires thoroughly understanding Pythonic scope, and the mutable default argument spandrel demonstrates how we can do closure-like things in Python without needing traditional lexical closures.

If you think of any more ways to solve this puzzle or find anything I've said here to be incorrect, or have any other comments, please email jurgen at botz.org.